# Model orchestration

The following notebook shows how to generate metadata for the model, including ethnicity data that will be used to ensure the model is not unfairly biased. It installs the [Surgeo](https://github.com/theonaunheim/surgeo) library, an open source implementation of the Bayesian Improved Surname Geocode (BISG) model. Begin by pasting your Cloud Pak for Data admin password and the GUID for the pre-production deployment space in the cell below, then run the notebook one cell at a time, stopping to take action where required.

In [ ]:
CPD_USERNAME = "admin"
CPD_PASSWORD = "PASSWORD"
EXPERIMENT_NAME = "AutoPolicyRiskFactSheet"
SPACE_GUID = "GUID"

Get information about the environment.

In [ ]:
import os

CPD_URL = os.environ['RUNTIME_ENV_APSX_URL']
CONTAINER_ID = SPACE_GUID
CONTAINER_TYPE = "space"

WML_CREDENTIALS = {
    "url": CPD_URL,
    "password": CPD_PASSWORD,
    "username": CPD_USERNAME,
    "instance_id": "wml_local",
    "version": "4.5"
}

Create a new instance of the Watson Machine Learning client.

In [ ]:
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
print(wml_client.version)

Set the deployment space with the model to the default space for the WML client, so the models can be scored using the Python client.

In [ ]:
wml_client.set.default_space(SPACE_GUID)

List the deployed models in the space.

In [ ]:
wml_client.deployments.list()

# Action: copy and paste the GUID of the deployed model

Copy and paste the GUID of the model from the output above into the cell below, which will look something like this:
 ```
 DEPLOYMENT_UID = '364ecd27-4b79-4c93-bc68-c7993a43244d'
 ```

In [ ]:
DEPLOYMENT_UID = 'GUID'

Get the evaluation data.

In [ ]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

import pandas as pd

df_data_1 = pd.read_csv(wslib.mount.get_data_path('policy_risk_evaluation.csv'))
df_data_1.head()


Select five records at random from the data, and remove the label column.

In [ ]:
scoring_df = df_data_1.sample(n=5).drop(['RISK'], axis=1)
scoring_df.head()

Ensure that the surgeo library is available.

In [ ]:
try:
    import surgeo
except:
    !pip install -U surgeo
    import surgeo

Create a new instance of the surgeo model.

In [ ]:
sg = surgeo.SurgeoModel()

Get the last names and zip codes of the sample data as pandas series.

In [ ]:
names = pd.Series(scoring_df['LAST_NAME'].values)
zips = pd.Series(scoring_df['ZIPCODE'].values)

Use the surgeo library to generate probably ethnicities for the policy holders in the sample data.

In [ ]:
sg_results = sg.get_probabilities(names, zips)
sg_results

Define a function to determine if the policy holder is the member of a minority group. For simplicity's sake, assume that if the probability that they are white (Caucasian) is less than 50%, then they are a minority. Note that this is US-specific, as the training data is for a US city.

In [ ]:
def assign_value(percent):
    if percent > 0.5:
        return 'NON-MINORITY'
    return 'MINORITY'

Apply the defined function to each row of the results.

In [ ]:
sg_results["MINORITY"] = sg_results.apply(lambda row: assign_value(row["white"]), axis=1)
sg_results.head()

Add the results to the scoring data.

In [ ]:
scoring_df['MINORITY'] = sg_results['MINORITY'].values
scoring_df.head()

Define the metadata for the model, and remove it from the scoring data.

In [ ]:
meta_df = scoring_df[['HOUSEHOLD_ID', 'POLICY_ID', 'LAST_NAME', 'MINORITY']]
scoring_df.drop(['HOUSEHOLD_ID', 'POLICY_ID', 'LAST_NAME', 'MINORITY', 'ZIPCODE'], axis=1, inplace=True)

Format the data for submission to the model. Submitting the metadata in this format will ensure it is captured by OpenScale, allowing the model to be evaluated for unfair bias.

In [ ]:
fields = scoring_df.columns.tolist()
values = scoring_df[fields].values.tolist()

meta_fields = meta_df.columns.tolist()
meta_values = meta_df[meta_fields].values.tolist()

meta_payload = {
    "fields": meta_fields,
    "values": meta_values
}

payload = {"input_data": [{"fields": fields, "values": values, "meta": meta_payload}]}

Submit the data to the model for scoring.

In [ ]:
scoring_response = wml_client.deployments.score(DEPLOYMENT_UID, payload)

In [ ]:
scoring_response